# Feature Selection for log_diff_model (Multivariate Model)

In [23]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from bdi_predict.ml_logic.sequence_gen import WindowGenerator
from tensorflow.keras import layers
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
bdi = pd.read_csv("../raw_data/data/BDI/log_diff_BDI_daily.csv")
bdi.head(2)

,Unnamed: 0,Date,Price,abs_price,log_price,log_diff
0,0,1995-01-03,1964.0,NaN,3.293141,NaN
1,1,1995-01-04,1961.0,-3.0,3.292478,-0.000664


In [25]:
bdi["Date"] = pd.to_datetime(bdi["Date"])
bdi.set_index("Date", inplace=True)
bdi= bdi[bdi.index != "1995-01-03"]
bdi.drop(columns="Unnamed: 0", inplace=True)
bdi.head(2)

,Price,abs_price,log_price,log_diff
Date,,,,
1995-01-04,1961.0,-3.0,3.292478,-0.000664
1995-01-05,1967.0,6.0,3.293804,0.001327


In [35]:
libor

,Year,Value
0,1995-01-01,7.78
1,1995-02-01,7.47
2,1995-03-01,7.20
3,1995-04-01,7.06
4,1995-05-01,6.63
...,...,...
329,2022-06-01,3.14
330,2022-07-01,2.90
331,2022-08-01,2.90
332,2022-09-01,3.52


In [40]:
libor

,Year,Value
0,1995-01-01,7.78
1,1995-02-01,7.47
2,1995-03-01,7.20
3,1995-04-01,7.06
4,1995-05-01,6.63
...,...,...
329,2022-06-01,3.14
330,2022-07-01,2.90
331,2022-08-01,2.90
332,2022-09-01,3.52


In [44]:
libor= pd.read_csv("../raw_data/data/LIBOR/cleaned_interest_rate.csv")
libor.rename(columns={"Year":"Date"}, inplace=True)
libor["Date"] = pd.to_datetime(libor["Date"])
libor.set_index("Date", inplace=True)
libor.head(3)

,Value
Date,
1995-01-01,7.78
1995-02-01,7.47
1995-03-01,7.20


In [135]:
datetime_index = pd.date_range(start="1995-01-01", end="2021-12-01", name="Date")

In [136]:
libor_daily = pd.DataFrame(index=datetime_index)

In [137]:
libor_daily["libor"] = libor["Value"]
libor_daily

,libor
Date,
1995-01-01,7.78
1995-01-02,NaN
1995-01-03,NaN
1995-01-04,NaN
1995-01-05,NaN
...,...
2021-11-27,NaN
2021-11-28,NaN
2021-11-29,NaN


In [163]:
libor_daily.interpolate(method="polynomial", order=2, inplace=True)
len(libor_daily)

9832

In [164]:
len(bdi)

7065

In [165]:
feat = pd.read_csv("../raw_data/data/merged_features.csv")
feat["Date"] = pd.to_datetime(feat["Date"])
feat.set_index("Date", inplace=True)
feat.drop(columns="Unnamed: 0", inplace=True)
feat = feat[["equally weighted"]]
feat.rename(columns={"equally weighted":"GHP"}, inplace=True)
GHP=feat
GHP[GHP.index > "2007-01-03"]

,GHP
Date,
2007-01-04,139.512036
2007-01-05,139.545038
2007-01-08,139.578039
2007-01-09,139.611040
2007-01-10,139.644041
...,...
2022-11-08,107.030433
2022-11-09,106.747531
2022-11-10,106.464629


In [167]:
df = pd.merge(bdi, libor_daily, on="Date", how="inner")
#df.drop(columns="Price_y", inplace=True)
len(df)

6817

In [168]:
df.head(3)

,Price,abs_price,log_price,log_diff,libor
Date,,,,,
1995-01-04,1961.0,-3.0,3.292478,-0.000664,7.751225
1995-01-05,1967.0,6.0,3.293804,0.001327,7.741574
1995-01-06,1983.0,16.0,3.297323,0.003518,7.731895


# Data Cleaning

## Holdout Method (manual, chronological)

In [169]:
input_length = len(df)

In [170]:
def train_test_split(df:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> tuple:
    '''
    Returns a train dataframe and a test dataframe (df_train, df_test)
    from which one can sample (X,y) sequences using TimeseriesGenerator.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    
    # TRAIN SET

    last_train_idx = round(train_test_ratio * len(df))
    df_train = df.iloc[0:last_train_idx, :]

    # TEST SET
 
    first_test_idx = last_train_idx - input_length
    df_test = df.iloc[first_test_idx:, :]

    return (df_train, df_test)

In [171]:
df_train, df_test = train_test_split(df=df, train_test_ratio=0.8, input_length=input_length)

In [175]:
X_train = df_train[["Price","libor"]]
y_train = df_train["log_diff"]

X_test  = df_test[["Price","libor"]]
y_test = df_test["log_diff"]

In [176]:
#Feature Scaling

scaler_X = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.fit_transform(X_test)



In [177]:
generator = TimeseriesGenerator(X_train_scaled, y_train, length=20, batch_size=16, sampling_rate=1, stride=1)

In [178]:
len(generator)

340

In [179]:
generator_val = TimeseriesGenerator(X_test_scaled, y_test, length=20, batch_size=16, sampling_rate=1, stride=1)

In [180]:
for X, y in generator:
    print(X.shape, y.shape)
    print(X[0], y[0])
    #listy = [float(X[0][i]) for i in range(len(X[0]))]
    #print(listy)
    break

(16, 20, 2) (16,)
[[0.14519214 1.        ]
 [0.14571379 0.99845697]
 [0.14710485 0.99690927]
 [0.14806121 0.99223821]
 [0.14988698 0.99067186]
 [0.15023474 0.98910086]
 [0.15188663 0.98752519]
 [0.15379934 0.98594486]
 [0.15440793 0.98117589]
 [0.15475569 0.97957692]
 [0.15475569 0.97797327]
 [0.15466875 0.97636497]
 [0.1537124  0.97475201]
 [0.15240828 0.96988514]
 [0.15136498 0.96825353]
 [0.1501478  0.96661725]
 [0.14823509 0.96497632]
 [0.14710485 0.96333072]
 [0.1465832  0.95836595]
 [0.14623544 0.9567017 ]] -0.0015435717560743


# The Model

In [181]:
# Learning Rate Scheduler: 

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)

In [182]:
#Optimizers

rmsprop = RMSprop(learning_rate=lr_schedule) #or 0.001 for inital test
adam = Adam(learning_rate=lr_schedule)

In [183]:
#Early Stopping Criterion:

es = EarlyStopping(monitor="val_mae", patience=100, restore_best_weights=True)

In [184]:
#Initializing and compiling model:

def init_model():
    
    model = Sequential()

    #LSTM layers
    model.add(layers.LSTM(60, activation="tanh", input_shape=(20,2), return_sequences=False))

    #Dense layers
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(1, activation="linear"))

    #compiling model
    model.compile(loss="mse", optimizer=rmsprop, metrics="mae")
    
    return model

# Training and Testing

In [185]:
#Instantiating model

model = init_model()

In [186]:
#Model layers & params overview:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60)                15120     
                                                                 
 dense_12 (Dense)            (None, 100)               6100      
                                                                 
 dense_13 (Dense)            (None, 1)                 101       
                                                                 
Total params: 21,321
Trainable params: 21,321
Non-trainable params: 0
_________________________________________________________________


In [187]:
#TRAINING THE MODEL:

history = model.fit(
    generator, epochs=1000, validation_data=generator_val, shuffle=True, callbacks=es
)

Epoch 1/1000
340/340 [==============================] - 2s 4ms/step - loss: 2.2678e-04 - mae: 0.0107 - val_loss: 2.2723e-04 - val_mae: 0.0128
Epoch 2/1000
340/340 [==============================] - 1s 2ms/step - loss: 1.0448e-04 - mae: 0.0076 - val_loss: 1.4034e-04 - val_mae: 0.0095
Epoch 3/1000
340/340 [==============================] - 1s 2ms/step - loss: 8.3167e-05 - mae: 0.0066 - val_loss: 9.6515e-05 - val_mae: 0.0075
Epoch 4/1000
340/340 [==============================] - 1s 2ms/step - loss: 6.7408e-05 - mae: 0.0059 - val_loss: 9.6819e-05 - val_mae: 0.0074
Epoch 5/1000
340/340 [==============================] - 1s 2ms/step - loss: 6.8237e-05 - mae: 0.0058 - val_loss: 1.0602e-04 - val_mae: 0.0079
Epoch 6/1000
340/340 [==============================] - 1s 2ms/step - loss: 6.4013e-05 - mae: 0.0055 - val_loss: 9.3938e-05 - val_mae: 0.0074
Epoch 7/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.9248e-05 - mae: 0.0052 - val_loss: 9.1713e-05 - val_mae: 0.0074
Epoch 

340/340 [==============================] - 1s 2ms/step - loss: 5.7425e-05 - mae: 0.0051 - val_loss: 9.4517e-05 - val_mae: 0.0075
Epoch 59/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7327e-05 - mae: 0.0051 - val_loss: 8.9449e-05 - val_mae: 0.0071
Epoch 60/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7277e-05 - mae: 0.0051 - val_loss: 9.0158e-05 - val_mae: 0.0072
Epoch 61/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7667e-05 - mae: 0.0051 - val_loss: 9.0471e-05 - val_mae: 0.0072
Epoch 62/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7910e-05 - mae: 0.0051 - val_loss: 8.9482e-05 - val_mae: 0.0071
Epoch 63/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.8307e-05 - mae: 0.0051 - val_loss: 8.9452e-05 - val_mae: 0.0071
Epoch 64/1000
340/340 [==============================] - 1s 3ms/step - loss: 5.7817e-05 - mae: 0.0051 - val_loss: 8.9592e-05 - val_mae: 0.0071
Epoch 65/1000

340/340 [==============================] - 1s 2ms/step - loss: 5.7063e-05 - mae: 0.0051 - val_loss: 8.9457e-05 - val_mae: 0.0071
Epoch 116/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7376e-05 - mae: 0.0051 - val_loss: 9.5698e-05 - val_mae: 0.0074
Epoch 117/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7340e-05 - mae: 0.0051 - val_loss: 8.9616e-05 - val_mae: 0.0071
Epoch 118/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6989e-05 - mae: 0.0050 - val_loss: 9.0779e-05 - val_mae: 0.0073
Epoch 119/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7687e-05 - mae: 0.0051 - val_loss: 9.2481e-05 - val_mae: 0.0074
Epoch 120/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7198e-05 - mae: 0.0051 - val_loss: 9.0685e-05 - val_mae: 0.0072
Epoch 121/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7458e-05 - mae: 0.0051 - val_loss: 8.9901e-05 - val_mae: 0.0072
Epoch 1

Epoch 172/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6507e-05 - mae: 0.0050 - val_loss: 9.1712e-05 - val_mae: 0.0072
Epoch 173/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.7132e-05 - mae: 0.0051 - val_loss: 9.0676e-05 - val_mae: 0.0072
Epoch 174/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6544e-05 - mae: 0.0050 - val_loss: 9.0691e-05 - val_mae: 0.0072
Epoch 175/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6939e-05 - mae: 0.0050 - val_loss: 9.1533e-05 - val_mae: 0.0073
Epoch 176/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6681e-05 - mae: 0.0050 - val_loss: 9.1149e-05 - val_mae: 0.0073
Epoch 177/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6821e-05 - mae: 0.0051 - val_loss: 8.9460e-05 - val_mae: 0.0071
Epoch 178/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6693e-05 - mae: 0.0051 - val_loss: 8.9838e-05 - val_mae:

Epoch 229/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6782e-05 - mae: 0.0050 - val_loss: 8.9566e-05 - val_mae: 0.0072
Epoch 230/1000
340/340 [==============================] - 1s 3ms/step - loss: 5.6772e-05 - mae: 0.0050 - val_loss: 8.9454e-05 - val_mae: 0.0071
Epoch 231/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6617e-05 - mae: 0.0050 - val_loss: 8.9572e-05 - val_mae: 0.0072
Epoch 232/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6583e-05 - mae: 0.0050 - val_loss: 8.9763e-05 - val_mae: 0.0071
Epoch 233/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6707e-05 - mae: 0.0050 - val_loss: 9.1841e-05 - val_mae: 0.0073
Epoch 234/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6861e-05 - mae: 0.0050 - val_loss: 8.9466e-05 - val_mae: 0.0071
Epoch 235/1000
340/340 [==============================] - 1s 2ms/step - loss: 5.6514e-05 - mae: 0.0050 - val_loss: 9.2829e-05 - val_mae:

In [ ]:
model.evaluate(generator_val)

In [ ]:
# Learning Curves:
plot_history(history)

In [20]:
#Baseline MAE
0.0048812746058099115

0.0048812746058099115

# Learning Curves viz code:

In [21]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # Loss:MSE
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # Metrics:MAE
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax